<a href="https://colab.research.google.com/github/TXH2020/MainRepo/blob/main/Academics/AI%20Project/chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request as ureq
ureq.urlretrieve('https://raw.githubusercontent.com/TXH2020/MainRepo/main/Academics/AI%20Project/intents.json','intents.json')

('intents.json', <http.client.HTTPMessage at 0x7f2c4e575290>)

In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')
# things we need for Tensorflow
import numpy as np
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

78 documents
26 classes ['Delivery', 'FAQ', 'Gifts', 'Insurance', 'LuckyDraw', 'Seconds', 'branches', 'details', 'discount', 'goodbye', 'greeting', 'holiday', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'specs1', 'specs2', 'specs3', 'thanks', 'today', 'type1', 'type2', 'type3', 'warranty']
134 unique stemmed words ["'d", "'m", "'s", 'a', 'about', 'acceiv', 'aft', 'afternoon', 'al', 'allow', 'am', 'and', 'any', 'anyon', 'ar', 'as', 'avail', 'be', 'branch', 'brief', 'bye', 'can', 'card', 'cash', 'city', 'clos', 'colo', 'company', 'conduc', 'coupon', 'cov', 'credit', 'custom', 'day', 'del', 'delivery', 'design', 'detail', 'discount', 'do', 'doe', 'draw', 'dur', 'elig', 'ev', 'feedback', 'fin', 'for', 'get', 'gift', 'giv', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'holiday', 'hom', 'hour', 'how', 'i', 'in', 'inform', 'ins', 'is', 'it', 'kind', 'lat', 'lik', 'look', 'lucky', 'lunch', 'many', 'mastercard', 'me', 'model', 'mop', 'mor', 'morn', 'netbank', 'of', 'off', 'on', 'op

In [5]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [6]:
# reset underlying graph data
import tensorflow as tf
from tensorflow import keras

model=keras.Sequential([
    keras.layers.Dense(8,input_shape=(len(train_x[0]),),kernel_initializer='normal'),
    keras.layers.Dense(8,kernel_initializer='normal'),
    keras.layers.Dense(len(train_y[0]),activation='softmax',kernel_initializer='normal')
])
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y), epochs=1000, batch_size=8, verbose=1)

Epoch 1/1000
10/10 [==============================] - 1s 2ms/step - loss: 0.0179 - accuracy: 0.0256
Epoch 2/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0179 - accuracy: 0.0897
Epoch 3/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0179 - accuracy: 0.1026
Epoch 4/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0179 - accuracy: 0.1026
Epoch 5/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0179 - accuracy: 0.1154
Epoch 6/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0179 - accuracy: 0.1154
Epoch 7/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0179 - accuracy: 0.1154
Epoch 8/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0179 - accuracy: 0.1282
Epoch 9/1000
10/10 [==============================] - 0s 2ms/step - loss: 0.0178 - accuracy: 0.1410
Epoch 10/1000
10/10 [==============================] - 0s 3ms/step - loss: 0.0178 - accuracy: 0.1667

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [8]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
['Delivery', 'FAQ', 'Gifts', 'Insurance', 'LuckyDraw', 'Seconds', 'branches', 'details', 'discount', 'goodbye', 'greeting', 'holiday', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'specs1', 'specs2', 'specs3', 'thanks', 'today', 'type1', 'type2', 'type3', 'warranty']


In [9]:
print(model.predict(np.array([p])))

1/1 [==============================] - 0s 85ms/step
[[1.1124453e-11 2.2906969e-08 4.5964781e-11 1.1844975e-10 2.8885547e-06
  1.2133769e-03 4.0343592e-10 8.1085682e-07 4.8741626e-05 4.2927579e-04
  5.1972677e-04 3.2409604e-04 2.4228400e-01 4.0451285e-11 7.5511378e-01
  4.6873069e-05 9.7416887e-06 2.2503367e-11 6.1525477e-12 9.0422374e-12
  2.6233415e-09 3.7006518e-07 2.1954847e-07 2.2143754e-11 1.8715682e-11
  6.1120563e-06]]


In [10]:
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict(np.array([bow(sentence, words)]))[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [11]:
classify('is your shop open today?')

1/1 [==============================] - 0s 17ms/step


[('opentoday', 0.7551138)]

In [12]:
response('is your shop open today?')

1/1 [==============================] - 0s 18ms/step
Our hours are 9am-9pm every day


In [13]:
response('do you take cash?')

1/1 [==============================] - 0s 17ms/step
Netbanking is allowed


In [14]:
response('what kind of mopeds do you rent?')

1/1 [==============================] - 0s 17ms/step
We have Piaggio, Vespa and Yamaha mopeds. Which type you are looking for?


In [15]:
response('Goodbye, see you later')

1/1 [==============================] - 0s 19ms/step
See you later, thanks for visiting


In [16]:
context

{'123': 'models'}

In [17]:
response('we want to rent a moped')

1/1 [==============================] - 0s 19ms/step
Rental services are available on Monday, Wednesday and Friday. R u looking for today?


In [18]:
context

{'123': 'rentalday'}

In [19]:
response('today')

1/1 [==============================] - 0s 18ms/step
For rentals, please call 1-800-23415423


In [20]:
classify('today')

1/1 [==============================] - 0s 17ms/step


[('today', 0.9921457)]

In [21]:
response("Hi there!", show_details=True)

1/1 [==============================] - 0s 18ms/step
context: 
tag: greeting
Good to see you again


In [22]:
response('today')
classify('today')

1/1 [==============================] - 0s 18ms/step


[('today', 0.9921457)]

In [23]:
response("thanks, your great")

1/1 [==============================] - 0s 18ms/step
Any time!
